In [ ]:
from __future__ import print_function, division
import itertools
from copy import deepcopy
from collections import OrderedDict
from warnings import warn

import pandas as pd
import numpy as np
from hmmlearn import hmm

from nilmtk.feature_detectors import cluster
from nilmtk.disaggregate import Disaggregator

SEED = 42

# Fix the seed for repeatibility of experiments
np.random.seed(SEED)


class FHMM_across_homes(Disaggregator):
    """
    Attributes
    ----------
    model : dict
    predictions : pd.DataFrame()
    meters : list
    MIN_CHUNK_LENGTH : int
    """

    def __init__(self):
        self.model = {}
        self.predictions = pd.DataFrame()
        self.MIN_CHUNK_LENGTH = 100
        self.MODEL_NAME = 'FHMM'

    def train(self, list_of_homes, list_of_appliances, min_activation=0.05, **load_kwargs):
        """Train using 1d FHMM.

        Places the learnt model in `model` attribute
        The current version performs training ONLY on the first chunk.
        Online HMMs are welcome if someone can contribute :)
        Assumes all pre-processing has been done.
        """
        models = {}

        for appliance in list_of_appliances:
            print appliance, "training"
            o = []
            for building_num in list_of_homes:

                building = ds.buildings[building_num]
                elec = building.elec
                try:
                    df = elec[appliance].load(**load_kwargs).next().squeeze()
                    appl_power = df.dropna().values.reshape(-1,1)
                    activation = (df>10).sum()*1.0/len(df)
                    if activation>min_activation:
                        o.append(appl_power)
                except:
                    pass

            if len(o)>1:
                o = np.array(o)
                mod = hmm.GaussianHMM(2, "full")
                mod.fit(o)
                models[appliance] = mod
                print "Means for %s are" %appliance
                print mod.means_
            else:
                print "Not enough samples for %s" %appliance
        
        self.individual = new_learnt_models
        self.model = learnt_model_combined

    def disaggregate_chunk(self, test_mains):
        """Disaggregate the test data according to the model learnt previously

        Performs 1D FHMM disaggregation.

        For now assuming there is no missing data at this stage.
        """
        # See v0.1 code
        # for ideas of how to handle missing data in this code if needs be.

        # Array of learnt states
        learnt_states_array = []
        test_mains = test_mains.dropna()
        length = len(test_mains.index)
        temp = test_mains.values.reshape(length, 1)
        learnt_states_array.append(self.model.predict(temp))

        # Model
        means = OrderedDict()
        for elec_meter, model in self.individual.iteritems():
            means[elec_meter] = (
                model.means_.round().astype(int).flatten().tolist())
            means[elec_meter].sort()

        decoded_power_array = []
        decoded_states_array = []

        for learnt_states in learnt_states_array:
            [decoded_states, decoded_power] = decode_hmm(
                len(learnt_states), means, means.keys(), learnt_states)
            decoded_states_array.append(decoded_states)
            decoded_power_array.append(decoded_power)

        prediction = pd.DataFrame(
            decoded_power_array[0], index=test_mains.index)

        return prediction

    def disaggregate(self, mains, output_datastore, **load_kwargs):
        '''Disaggregate mains according to the model learnt previously.

        Parameters
        ----------
        mains : nilmtk.ElecMeter or nilmtk.MeterGroup
        output_datastore : instance of nilmtk.DataStore subclass
            For storing power predictions from disaggregation algorithm.
        sample_period : number, optional
            The desired sample period in seconds.
        **load_kwargs : key word arguments
            Passed to `mains.power_series(**kwargs)`
        '''
        load_kwargs = self._pre_disaggregation_checks(load_kwargs)

        load_kwargs.setdefault('sample_period', 60)
        load_kwargs.setdefault('sections', mains.good_sections())

        timeframes = []
        building_path = '/building{}'.format(mains.building())
        mains_data_location = building_path + '/elec/meter1'
        data_is_available = False

        import warnings
        warnings.filterwarnings("ignore", category=Warning)

        for chunk in mains.power_series(**load_kwargs):
            # Check that chunk is sensible size before resampling
            if len(chunk) < self.MIN_CHUNK_LENGTH:
                continue

            # Record metadata
            timeframes.append(chunk.timeframe)
            measurement = chunk.name

            # Start disaggregation
            predictions = self.disaggregate_chunk(chunk)

            for meter in predictions.columns:
                meter_instance = meter.instance()
                cols = pd.MultiIndex.from_tuples([chunk.name])

                predicted_power = predictions[[meter]]
                if len(predicted_power) == 0:
                    continue
                data_is_available = True
                output_df = pd.DataFrame(predicted_power)
                output_df.columns = pd.MultiIndex.from_tuples([chunk.name])
                key = '{}/elec/meter{}'.format(building_path, meter_instance)
                output_datastore.append(key, output_df)

            # Copy mains data to disag output
            output_datastore.append(key=mains_data_location,
                                    value=pd.DataFrame(chunk, columns=cols))

        if data_is_available:
            self._save_metadata_for_disaggregation(
                output_datastore=output_datastore,
                sample_period=load_kwargs['sample_period'],
                measurement=measurement,
                timeframes=timeframes,
                building=mains.building(),
                meters=self.meters
            )

In [1]:
from hmmlearn import hmm

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires tomorrow

    After the trial mode has expired, if you want to use mkl thereafter,
    please purchase a license at http://continuum.io
    


In [6]:
import nilmtk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict
import warnings

In [7]:
warnings.filterwarnings("ignore")

In [8]:
from nilmtk import *
import os
import nilmtk

In [9]:
ds = DataSet("/Users/nipunbatra/Downloads/wikienergy-2.h5")

In [10]:
models = {}

for appliance in ["air conditioner", "fridge", "washer dryer", "spin dryer", 
                  "dishwasher", "microwave","furnace","washing machine"]:
    print appliance, "training"
    o = []
    for building_num in range(1, 190):
        
        building = ds.buildings[building_num]
        elec = building.elec
        try:
            df = elec[appliance].load(cols=[('power','active')], sample_period=600).next().squeeze()
            appl_power = df.dropna().values.reshape(-1,1)
            activation = (df>10).sum()*1.0/len(df)
            if activation>0.05:
                o.append(appl_power)
        except:
            pass
    
    if len(o)>1:
        o = np.array(o)
        mod = hmm.GaussianHMM(2, "full")
        mod.fit(o)
        models[appliance] = mod
        print "Means for %s are" %appliance
        print mod.means_
    else:
        print "Not enough samples for %s" %appliance

air conditioner training
Means for air conditioner are
[[    7.53691615]
 [ 1139.82086448]]
fridge training
Means for fridge are
[[ 172.06615343]
 [  59.94911461]]
washer dryer training
Means for washer dryer are
[[   0.36924346]
 [ 128.36346017]]
spin dryer training
Means for spin dryer are
[[ 404.68524686]
 [   0.93259225]]
dishwasher training
Not enough samples for dishwasher
microwave training
Means for microwave are
[[  4.07876205]
 [ 82.86502975]]
furnace training
Not enough samples for furnace
washing machine training
Means for washing machine are
[[   0.65830403]
 [ 229.74056184]]


In [11]:
from nilmtk.disaggregate.fhmm_exact import sort_learnt_parameters

In [12]:
new_learnt_models = OrderedDict()
for appliance, appliance_model in models.iteritems():
    startprob, means, covars, transmat = sort_learnt_parameters(
                    appliance_model.startprob_, appliance_model.means_,
                    appliance_model.covars_, appliance_model.transmat_)
    new_learnt_models[appliance] = hmm.GaussianHMM(
                startprob.size, "full", startprob, transmat)
    new_learnt_models[appliance].means_ = means
    new_learnt_models[appliance].covars_ = covars

In [13]:
from nilmtk.disaggregate.fhmm_exact import create_combined_hmm

In [14]:
learnt_model_combined = create_combined_hmm(new_learnt_models)

In [15]:
from nilmtk.disaggregate.fhmm_exact import FHMM

In [16]:
f = FHMM()

In [17]:
f.model = learnt_model_combined
f.individual = new_learnt_models

In [18]:
import pickle

In [19]:
pickle.dump(f, open( "../fhmm_model.p", "wb" ))

In [20]:
d = ds.buildings[191].elec

In [22]:
d

MeterGroup(meters=
  ElecMeter(instance=1, building=191, dataset='WikiEnergy', site_meter, appliances=[])
  ElecMeter(instance=2, building=191, dataset='WikiEnergy', appliances=[Appliance(type='air conditioner', instance=1)])
  ElecMeter(instance=3, building=191, dataset='WikiEnergy', appliances=[Appliance(type='spin dryer', instance=1)])
)

In [23]:
d = ds.buildings[191].elec.mains().load(cols=[('power','active')], sample_period=600).next()

In [67]:
preds = {}
for building in range(191, 240):
    print building
    preds[building] = []
    try:
        iterator_mains_data = ds.buildings[building].elec.mains().load(cols=[('power','active')], sample_period=600)
        for mains_df in iterator_mains_data:
            pred = f.disaggregate_chunk(mains_df)
            preds[building].append(pred)
        out_df = pd.concat(preds[building])
        out_df.to_csv("/Users/nipunbatra/Dropbox/ss_results/%d.csv" %building)
    except:
        pass
    


191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239


In [65]:
print building
ds.buildings[building].elec.mains().load(cols=[('power','active')], sample_period=600)

199


AttributeError: 'NoneType' object has no attribute 'load'

In [66]:
ds.buildings[199].elec.mains().load(cols=[('power','active')], sample_period=600).next()

AttributeError: 'NoneType' object has no attribute 'load'

In [37]:
%matplotlib inline

In [42]:
a = ds.buildings[193].elec

In [45]:
a.submeters().load()

MeterGroup(meters=
  ElecMeter(instance=2, building=193, dataset='WikiEnergy', appliances=[Appliance(type='air conditioner', instance=1)])
  ElecMeter(instance=3, building=193, dataset='WikiEnergy', appliances=[Appliance(type='spin dryer', instance=1)])
)

In [256]:
print (pred['refrigerator1'].sum()-df_26['refrigerator1'].sum())/df_26['refrigerator1'].sum()

-0.207079013041


In [145]:
mod.fit([o[0].reshape(-1,1)])

GaussianHMM(algorithm='viterbi', covariance_type='full', covars_prior=0.01,
      covars_weight=1,
      init_params='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ',
      means_prior=None, means_weight=0, n_components=2, n_iter=10,
      params='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ',
      random_state=None, startprob=None, startprob_prior=1.0, thresh=0.01,
      transmat=None, transmat_prior=1.0)

In [146]:
mod.means_

array([[   0.        ],
       [ 987.96043864]])

In [60]:

startprob = np.array([0.6, 0.3, 0.1, 0.0])
# The transition matrix, note that there are no transitions possible
# between component 1 and 3
transmat = np.array([[0.7, 0.2, 0.0, 0.1],
                     [0.3, 0.5, 0.2, 0.0],
                     [0.0, 0.3, 0.5, 0.2],
                     [0.2, 0.0, 0.2, 0.6]])
# The means of each component
means = np.array([[0.0,  0.0],
                  [0.0, 11.0],
                  [9.0, 10.0],
                  [11.0, -1.0]])
# The covariance of each component
covars = .5 * np.tile(np.identity(2), (4, 1, 1))

# Build an HMM instance and set parameters
model = hmm.GaussianHMM(n_components=4, covariance_type="full",
                        random_state=42)

# Instead of fitting it from the data, we directly set the estimated
# parameters, the means and covariance of the components
model.startprob_ = startprob
model.transmat_ = transmat
model.means_ = means
model.covars_ = covars
###############################################################

# Generate samples
X, Z = model.sample(500)

In [64]:
model.fit([X])

GaussianHMM(algorithm='viterbi', covariance_type='full', covars_prior=0.01,
      covars_weight=1,
      init_params='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ',
      means_prior=None, means_weight=0, n_components=4, n_iter=10,
      params='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ',
      random_state=42, startprob=None, startprob_prior=1.0, thresh=0.01,
      transmat=None, transmat_prior=1.0)

In [67]:
model.transmat_

array([[  7.32035166e-01,   1.47944834e-01,   1.01478454e-01,
          1.85415463e-02],
       [  1.76165797e-01,   7.25388601e-01,   9.84454431e-02,
          1.58890415e-07],
       [  3.42950776e-02,   2.47572979e-01,   6.16853483e-01,
          1.01278460e-01],
       [  7.03806603e-01,   6.67735612e-08,   2.95388532e-01,
          8.04797824e-04]])

In [24]:
%matplotlib inline

In [26]:
df['refrigerator1']['2013'].resample("15T").head(10000).plot()

KeyError: 'refrigerator1'